# Examen Final

- Prender cámaras.
- Puede utilizar sus apuntes.
- Puede utilizar los notebooks del curso.
- La alumna o alumno que sea sorprendido solicitando ayuda a cualquier persona, sea ella compañera o no, será calificada con un 2.
- Cada pregunta vale un punto.
- La nota mínima es un 2.
- La nota se determinará con la siguiente fórmula $nota = \min\left(puntos + 2, 7\right)$
- Todas las funciones necesarias para resolver los ejercicios están ya importadas al notebook.

## Librerías

In [1]:
from finrisk import examen_final as ef
from scipy.interpolate import interp1d
import modules.hull_white as hw
import pandas as pd
import numpy as np
import textwrap
import random
import math

## Curva Cero Cupón

Los plazos están en días y las tasas en convención exp act/365.

In [2]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [3]:
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [4]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')

## Cálculo Opción

Para identificar si es una Call o una Put se usa un `enum`.

In [5]:
c_p = hw.CallPut.CALL
c_p = hw.CallPut.PUT

Ver toda la documentación:

In [6]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


## Valores `gamma` y `sigma`

In [7]:
gamma = 1
sigma = .005

In [8]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


## Preguntas

In [9]:
try:
    q = ef.get_questions()
except Exception as e:
    print(str(e))

In [10]:
for qq in q:
    print(f'{qq[0]}\n')
    print(textwrap.fill(f'{qq[1]}', 80))
    print('\n')

Pregunta 1:

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.


Pregunta 2:

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y número aleatorio N(0, 1) igual a .6574


Pregunta 3:

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,020.00 (365 días después).


Pregunta 4:

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 99.0%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.


Pregunta 5:

Ceteris paribus una opción sobre una activo con reversión a la media vale menos
que una opción sobre un activo sin reversión a la media. Comente.


Pregunta 6:

Considere un swap a 1Y con cupones semestrales de 1.3% (lineal). 

## Respuestas

### Pregunta 1

In [47]:
print("La diferencia entre usar la medida libre de riesgo y la medida forward es que en el caso de Forward hay que calcular un drift de theta, por otro lado también difieren en la forma de traer a valor presente ya que en el caso de Forward el payoff se multiplica por el factor de descuento en T, y en el caso de libre de riesgo el valor presente se calcula descontando en base a la curva")

La diferencia entre usar la medida libre de riesgo y la medida forward es que en el caso de Forward hay que calcular un drift de theta, por otro lado también difieren en la forma de traer a valor presente ya que en el caso de Forward el payoff se multiplica por el factor de descuento en T, y en el caso de libre de riesgo el valor presente se calcula descontando en base a la curva


### Pregunta 2

In [49]:
from typing import List, Tuple
import scipy.optimize as opt
gamma = 1
sigma = .005
r_ = .01
r0 = 0.000811
def vasicek_path(r0: float,
                 gamma: float, r_: float,
                 sigma: float,
                 num_dias: int = 1,
                 dias_agno: int = 264,
                 pasos_dia: int = 1) -> List[Tuple[float, float]]:

    dt = 1 / (dias_agno * pasos_dia)
    dt_gamma_r_ = dt * gamma * r_
    sigma_sqdt = sigma * math.sqrt(dt)
    result = [(0, r0),]
    gamma_dt = gamma * dt
    
    r = r0
    for i in range(1, (num_dias + 1) * pasos_dia):
        r = dt_gamma_r_ + (1 - gamma_dt) * r + sigma_sqdt * 0.6574 #random asignado
        result.append((i * dt, r))
    return result
sim = vasicek_path(r0, gamma, r_, sigma, num_dias=1)
print(sim)

[(0, 0.000811), (0.003787878787878788, 0.0010481076836063516)]


### Pregunta 3

In [50]:
ICPo = 10000
ICP365 = 10020
t = 365
Tna = ((ICP365/ICPo)-1)*(360/t)
round(Tna,4)

0.002

### Pregunta 4

In [51]:
strikes =[.99]
for k in strikes:
    print(hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.0095605971116699


### Pregunta 5

In [48]:
print("Un activo que tiene reversión a la media presenta menor volatilidad que otro que no cumple esto y que por lo tanto puede seguir una tendencia, por lo que la valorización de una opción de aun activo subyacente con menor volatilidad es menor a la de uno con mayor volatilidad") 

Un activo que tiene reversión a la media presenta menor volatilidad que otro que no cumple esto y que por lo tanto puede seguir una tendencia, por lo que la valorización de una opción de aun activo subyacente con menor volatilidad es menor a la de uno con mayor volatilidad


### Pregunta 6

### Pregunta 7

In [52]:
CRV=interp1d(df_curva["plazo"],df_curva["tasa"],"linear")
T1=982
T2=1454
R1=CRV(T1)
R2=CRV(T2)
DF1=math.exp(-R1*T1/365)
DF2=math.exp(-R2*T2/365)
Forward=(DF1/DF2-1.0)*360.0/(T2-T1)
print(Forward)

0.002368765983049694
